## Chạy đoạn này mỗi khi mở file

In [7]:
import time
import traceback
from solver_helper import preprocess_raw_image, array_to_label, image_to_tensor, remove_grid, trim_border
import tensorflow as tf
import cv2
import numpy as np
from base64 import b64decode
import pandas as pd
from time import sleep
import urllib.request
import random, string
from bs4 import BeautifulSoup
import warnings
from time import localtime, strftime
from urllib.request import Request, urlopen 
import json
import regex
import urllib.parse
from zipfile import ZipFile
import urllib


import os

os.environ['WDM_SSL_VERIFY'] = '0'

warnings.filterwarnings('ignore')

solver = tf.keras.models.load_model(".\CNN5_v10_acc_98.h5")

## Import dữ liệu

In [8]:
ten_file = "./CHECK MST - p2.xlsx"
list_mst = pd.read_excel(ten_file, converters = {'MST': str})["MST"]

## MST

In [9]:
import requests
from bs4 import BeautifulSoup
import random
import base64
# all cookies received will be stored in the session object

# proxies_s1 = requests.get(url='https://raw.githubusercontent.com/TheSpeedX/SOCKS-List/master/http.txt').text.splitlines()
proxies_s2 = requests.get(url='https://raw.githubusercontent.com/ErcinDedeoglu/proxies/main/proxies/https.txt').text.splitlines()
proxy_list = list(set(proxies_s2))
def data_crawler(mst, proxied= True, fast = False):
    s = requests.Session()
    if proxied: 
      proxies = {'https': 'https://' + random.choice(proxy_list)}
      s.proxies.update(proxies)
      print(proxies)
    headers = {
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
      'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
      'Cache-Control': 'max-age=0',
      'Connection': 'keep-alive',
      'Content-Type': 'application/x-www-form-urlencoded',
      'Origin': 'https://tracuunnt.gdt.gov.vn',
      'Referer': 'https://tracuunnt.gdt.gov.vn/tcnnt/mstdn.jsp',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'same-origin',
      'Sec-Fetch-User': '?1',
      'Upgrade-Insecure-Requests': '1',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
      'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Windows"'
    }

    step1 = s.get('https://tracuunnt.gdt.gov.vn/tcnnt/mstdn.jsp', verify=False, headers=headers)
    if step1.status_code != 200:
        print('failed, code: ', step1.status_code)
        return pd.DataFrame()
    captcha_img_src = BeautifulSoup(step1.text, "html.parser").select_one('td > div > img')['src']

    def _process_captcha2():
        def _get_captcha_image():
            b64img = base64.b64encode(s.get('https://tracuunnt.gdt.gov.vn'+captcha_img_src).content).decode('utf-8')
            image = cv2.imdecode(np.frombuffer(b64decode(b64img), np.uint8),-1)
            img = image[:,:,-1]
            print(img.shape)
            return img
        image = _get_captcha_image()
        image = preprocess_raw_image(image)
        image = image_to_tensor(image)
        image = tf.image.resize_with_pad(image, 64, 128)
        answer = solver.predict(image)
        answer = ''.join(array_to_label(answer[0]))
        return answer

    payload = f'action=action&id=&page=1&mst={mst}&fullname=&address=&cmt=&captcha={_process_captcha2()}'
    step2 = s.post('https://tracuunnt.gdt.gov.vn/tcnnt/mstdn.jsp', verify=False, data=payload, headers=headers)
    if step2.status_code != 200:
        print('failed, code: ', step2.status_code)
        return pd.DataFrame()
    t = pd.read_html(str(BeautifulSoup(step2.text, "html.parser").select_one('.ta_border')))[0][:-1].head(1)
    if fast == False:
        payload2 = f'action=action&id={mst}&page=1&mst={mst}&fullname=&address=&cmt=&captcha='
        step3 = s.post('https://tracuunnt.gdt.gov.vn/tcnnt/mstdn.jsp', verify=False, data=payload2, headers=headers)
        if step3.status_code != 200:
            print('failed, code: ', step3.status_code)
            t["Tên người đại diện"] = None
            t["Tên giám đốc"] = None
            t["Tên kế toán trưởng"] = None
            t["Ngày mở MST"] = None
            t["Ngày đóng MST"] = None
            t["Địa chỉ trụ sở"] = None
        else: 
            _tx = pd.read_html(str(BeautifulSoup(step3.text, "html.parser").select_one('.ta_border')))[0]
            t["Tên người đại diện"] = _tx.iloc[11,1]
            t["Tên giám đốc"] = _tx.iloc[12,1]
            t["Tên kế toán trưởng"] = _tx.iloc[13,1]
            t["Ngày mở MST"] = _tx.iloc[0,3]
            t["Ngày đóng MST"] = _tx.iloc[0,5]
            t["Địa chỉ trụ sở"] = _tx.iloc[3,1]
        payload3 = f'tin={mst}'    
        step4 = s.post('https://tracuunnt.gdt.gov.vn/tcnnt/nganhkinhdoanh.jsp', verify=False, data=payload3, headers=headers)
        if step4.status_code != 200:
            print('failed, code: ', step4.status_code)
            t["Ngành nghề"] = None
        else: 
            t["Ngành nghề"] = str(pd.read_html(str(BeautifulSoup(step4.text, "html.parser").select_one('.ta_border')))[0].to_dict('records'))   
        step5 = s.post('https://tracuunnt.gdt.gov.vn/tcnnt/chinhanh.jsp', verify=False, data=payload3, headers=headers)
        if step5.status_code != 200:
            print('failed, code: ', step5.status_code)
            t["Chi nhánh"] = None
        else: 
            t["Chi nhánh"] = str(pd.read_html(str(BeautifulSoup(step5.text, "html.parser").select_one('.ta_border')))[0].to_dict('records'))
        step6 = s.post('https://tracuunnt.gdt.gov.vn/tcnnt/tructhuoc.jsp', verify=False, data=payload3, headers=headers)    
        if step6.status_code != 200:
            print('failed, code: ', step6.status_code)
            t["DV trực thuộc"] = None
        else: 
            t["DV trực thuộc"] = str(pd.read_html(str(BeautifulSoup(step6.text, "html.parser").select_one('.ta_border')))[0].to_dict('records'))    
    return t

In [10]:
list_mst_done = pd.DataFrame()
saveddata = pd.DataFrame()
try:
    list_mst_done= pd.read_csv("./temp_mst.csv", quoting=3, escapechar="^", converters = {'MST': str}).astype(str)["MST"].to_list()
    list_mst_ok = pd.read_csv("./temp_mst.csv", quoting=3, escapechar="^" , converters = {'MST': str},index_col=0)
    list_mst_ok.to_csv("./temp_mst_{}.csv".format(strftime("%Y_%m_%d_%H_%M_%S", localtime())))
    saveddata = saveddata._append(list_mst_ok)
    print('MST DONE:', len(list_mst_done))
except:
    print(traceback.format_exc())
    pass

i = 0
arrlength = list_mst_done if len(list_mst_done) > 0 else []
list_mst = list_mst.loc[(np.isin(list_mst.to_list(),arrlength)==False) & list_mst.str.fullmatch(r'[0-9]{10}(-[0-9]{3})?') & (np.isin(list_mst.str.len(),[10,14]) == True)].sample(frac=1).reset_index(drop=True)

for mst in list_mst.to_list():
    if mst not in list_mst_done:
          try:
            table = data_crawler(mst, False)
            table['orig_query'] = mst
            i+=1; 
            print("MST: ",mst, " PROCESS:", i, "/", sum(np.isin(list_mst.to_list(),arrlength) == False))
            if len(table) > 0:
                saveddata = pd.concat([saveddata, table])
                if i % 100 == 0:
                    saveddata.to_csv('./temp_mst.csv', quoting=3, escapechar="^")
          except: pass
saveddata.to_excel('./check_mst_cn_manual_{}.xlsx'.format(strftime("%Y_%m_%d_%H_%M_%S", localtime())))

Traceback (most recent call last):
  File "C:\Users\anhviettran\AppData\Local\Temp\ipykernel_12752\4126081174.py", line 4, in <module>
    list_mst_done= pd.read_csv("./temp_mst.csv", quoting=3, escapechar="^", converters = {'MST': str}).astype(str)["MST"].to_list()
  File "C:\Users\anhviettran\AppData\Roaming\Python\Python39\site-packages\pandas\io\parsers\readers.py", line 948, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\anhviettran\AppData\Roaming\Python\Python39\site-packages\pandas\io\parsers\readers.py", line 611, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Users\anhviettran\AppData\Roaming\Python\Python39\site-packages\pandas\io\parsers\readers.py", line 1448, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "C:\Users\anhviettran\AppData\Roaming\Python\Python39\site-packages\pandas\io\parsers\readers.py", line 1705, in _make_engine
    self.handles = get_handle(
  File "C:\Users\anhviettran\AppDat